# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download(['punkt', 'wordnet', 'stopwords'])


from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier


import pickle
import warnings
warnings.filterwarnings('ignore')




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# load data from database
engine = create_engine('sqlite:///messageCategory.db')
df = pd.read_sql_table('messageCategory',engine)
X = df['message']
y = df.drop(['id','message', 'original','genre'], axis =1)
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):

    
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text) 
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()    
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    clean_tokens_sw_removed = [w for w in clean_tokens if w not in stopwords.words("english")]
    
    
    return clean_tokens_sw_removed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [15]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier()))
    ])

In [16]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fbfd7170840>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_trainpred = pipeline.predict(X_train)

In [9]:
for i,col in enumerate(y_train.columns):
    print('Category: {}'.format(col))
    print(classification_report(y_trainpred[:,i], y_train[col]))
    test_accuracy = (y_trainpred[:,i] == y_train[col]).mean()
    print('Test accuracy ={}'.format(test_accuracy))


Category: related
             precision    recall  f1-score   support

          0       0.97      0.99      0.98      4544
          1       1.00      0.99      0.99     14996
          2       0.86      0.99      0.92       122

avg / total       0.99      0.99      0.99     19662

Test accuracy =0.9893703590682534
Category: request
             precision    recall  f1-score   support

          0       1.00      0.98      0.99     16560
          1       0.92      1.00      0.96      3102

avg / total       0.99      0.99      0.99     19662

Test accuracy =0.986674804190825
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19605
          1       0.66      1.00      0.80        57

avg / total       1.00      1.00      1.00     19662

Test accuracy =0.9985250737463127
Category: aid_related
             precision    recall  f1-score   support

          0       1.00      0.98      0.99     11748
          1       0.

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
parameters = {
              'clf__estimator__n_estimators':[10,20],
              'clf__estimator__n_jobs': [1]
             }
              

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose=3)

In [11]:
cv.fit(X_train,y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__n_estimators=10, clf__estimator__n_jobs=1 .......
[CV]  clf__estimator__n_estimators=10, clf__estimator__n_jobs=1, score=0.24046383887702166, total= 1.7min
[CV] clf__estimator__n_estimators=10, clf__estimator__n_jobs=1 .......


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV]  clf__estimator__n_estimators=10, clf__estimator__n_jobs=1, score=0.22754551927576036, total= 1.7min
[CV] clf__estimator__n_estimators=20, clf__estimator__n_jobs=1 .......


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.9min remaining:    0.0s


[CV]  clf__estimator__n_estimators=20, clf__estimator__n_jobs=1, score=0.24738073441155528, total= 2.1min
[CV] clf__estimator__n_estimators=20, clf__estimator__n_jobs=1 .......
[CV]  clf__estimator__n_estimators=20, clf__estimator__n_jobs=1, score=0.24453260095615909, total= 2.1min


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 10.5min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20], 'clf__estimator__n_jobs': [1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [12]:
cv.best_params_

{'clf__estimator__n_estimators': 20, 'clf__estimator__n_jobs': 1}

In [13]:
cv.best_estimator_

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [14]:
cv.best_score_

0.24595666768385718

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_traincv = cv.predict(X_train)
y_testcv = cv.predict(X_test)

In [16]:
# classification report on train set
print('Grid search train metrics')
for i,col in enumerate(y_train.columns):
    print('Category: {}'.format(col))
    print(classification_report(y_traincv[:,i], y_train[col]))
    cvaccuracy_train = (y_traincv[:,i] == y_train[col]).mean()
    print('Test accuracy ={}'.format(cvaccuracy_train))
    

Grid search train metrics
Category: related
             precision    recall  f1-score   support

          0       0.99      0.99      0.99      4594
          1       1.00      1.00      1.00     14930
          2       0.96      0.99      0.97       138

avg / total       1.00      1.00      1.00     19662

Test accuracy =0.995575221238938
Category: request
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     16389
          1       0.97      1.00      0.99      3273

avg / total       1.00      1.00      1.00     19662

Test accuracy =0.9954735021869596
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19593
          1       0.80      1.00      0.89        69

avg / total       1.00      1.00      1.00     19662

Test accuracy =0.9991353880581832
Category: aid_related
             precision    recall  f1-score   support

          0       1.00      0.99      1.00     

In [17]:
# classification report on test set
print('Grid Search test metrics')
for i,col in enumerate(y_test.columns):
    print('Category: {}'.format(col))
    print(classification_report(y_testcv[:,i], y_test[col]))
    cvaccuracy_test = (y_testcv[:,i] == y_test[col]).mean()
    print('Test accuracy ={}'.format(cvaccuracy_test))

Grid Search test metrics
Category: related
             precision    recall  f1-score   support

          0       0.44      0.67      0.53       997
          1       0.93      0.84      0.89      5544
          2       0.19      0.69      0.30        13

avg / total       0.86      0.82      0.83      6554

Test accuracy =0.8158376563930424
Category: request
             precision    recall  f1-score   support

          0       0.98      0.90      0.94      5927
          1       0.45      0.81      0.58       627

avg / total       0.93      0.89      0.90      6554

Test accuracy =0.8878547451937748
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      6554

Test accuracy =0.9951174855050351
Category: aid_related
             precision    recall  f1-score   support

          0       0.84      0.77      0.80     

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [9]:
#Let us now try the KNearest Neighbours classifier 

DTpipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',MultiOutputClassifier(DecisionTreeClassifier()))
    ])

In [10]:
DTpipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fbfd7170840>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'),
              n_jobs=1))],
 

In [11]:
DTpipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))])

In [13]:
y_DTtrainpred = DTpipeline.predict(X_train)

In [14]:
# classification report for KNN classifier
print('Grid Search Decision Tree train metrics')

for i,col in enumerate(y_train.columns):
    print('Category: {}'.format(col))
    print(classification_report(y_DTtrainpred[:,i], y_train[col]))
    DT_accuracy = (y_DTtrainpred[:,i] == y_train[col]).mean()
    print('Test accuracy for Decision Tree classifier is ={}'.format(DT_accuracy))

Grid Search Decision Tree train metrics
Category: related
             precision    recall  f1-score   support

          0       1.00      0.99      1.00      4648
          1       1.00      1.00      1.00     14864
          2       0.97      0.99      0.98       150

avg / total       1.00      1.00      1.00     19662

Test accuracy for Decision Tree classifier is =0.9975078832265283
Category: request
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     16320
          1       0.99      1.00      1.00      3342

avg / total       1.00      1.00      1.00     19662

Test accuracy for Decision Tree classifier is =0.9991353880581832
Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00     19570
          1       0.99      1.00      0.99        92

avg / total       1.00      1.00      1.00     19662

Test accuracy for Decision Tree classifier is =0.9999491404740107
Category: ai

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [18]:
#Exporting the best model, i,e the Random forest classifier
pickle.dump(pipeline, open('classifier.pickle','wb'))